In [1]:
!pip install lifelines
!pip install pyckmeans
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 11.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=4bdffa6af1e7fbebb24f40409e3dbffd17577c7954b72e5a3c2d7c92de7eca7f
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 908.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyckmeans: filename=pyckmeans-0.9.4-cp310-cp310-linux_x86_64.whl size=93218 sha256=e8642ed32196bc34ba3f57796fc5ea61d2338d46396641932626351a5cdd06b9
  Stored in directory: /root/.cache/pip/wheels/c3/83/bb/d0fc31d29f361aff1f400ca2fb2d32c98f900e1aa480229f99
Successfully built pyckmeans
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.2 MB

In [2]:
import numpy as np
import pandas as pd
from pyckmeans import CKmeans
import matplotlib.pyplot as plt

from lifelines.fitters.coxph_fitter import CoxPHFitter

from google.colab import drive

import wandb
import os

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# --- Load raw BRCA data ---
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/TCGA-BRCA_1079.zip

Mounted at /content/gdrive
Archive:  gdrive/MyDrive/TCGA-BRCA_1079.zip
  inflating: TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv  


In [5]:
def drop_elements_from_cluster(cluster, consensus_matrix):
  new_cluster = []
  for j in range(0, len(cluster)):
    if consensus_matrix[cluster[0]][cluster[j]] == 1:
      new_cluster.append(cluster[j])
  return new_cluster

In [6]:
def convert_indices_to_binary_list(number_elements, cluster_0, cluster_1, cluster_2, cluster_3):
  cluster0_column = []
  cluster1_column = []
  cluster2_column = []
  cluster3_column = []
  for k in range(0, number_elements):
    if k in cluster_0:
      cluster0_column.append(1)
      cluster1_column.append(0)
      cluster2_column.append(0)
      cluster3_column.append(0)
    elif k in cluster_1:
      cluster0_column.append(0)
      cluster1_column.append(1)
      cluster2_column.append(0)
      cluster3_column.append(0)
    elif k in cluster_2:
      cluster0_column.append(0)
      cluster1_column.append(0)
      cluster2_column.append(1)
      cluster3_column.append(0)
    elif k in cluster_3:
      cluster0_column.append(0)
      cluster1_column.append(0)
      cluster2_column.append(0)
      cluster3_column.append(1)
    else:
      cluster0_column.append(0)
      cluster1_column.append(0)
      cluster2_column.append(0)
      cluster3_column.append(0)

  return [cluster0_column, cluster1_column, cluster2_column, cluster3_column]

In [7]:
def fit_cox_model(cluster, data_frame):
  print("Original dataframe shape: ", data_frame.shape)
  cluster_frame = pd.DataFrame({'cluster': cluster})
  # Concatenate cluster membership to data table
  if 'cluster' in data_frame.columns.values:
    data_frame = data_frame.drop('cluster', axis = 1) # Drop columns that could have been created previously
  data_frame = pd.concat([data_frame, cluster_frame], axis = 1)

  # Get rows with null entry
  null_mask = data_frame.isnull().any(axis=1)
  null_rows = data_frame[null_mask]
  print(null_rows)

  # Remove rows with null entry
  data_frame.dropna(inplace=True) # remove all rows with any null value
  print("Final dataframe shape: ", data_frame.shape)

  # Fit Cox model
  cph = CoxPHFitter()
  cph.fit(data_frame, duration_col = 'OS.time', event_col = 'OS')
  cph.print_summary()

  # exp(coef) <=> hazard ratio // AN ESTIMATE OF THE TRUE HAZARD RATIO. IT HAS A STANDARD ERROR ASSOCIATED WITH IT.
  # A one unit increase in the covariate will increase the hazard by the hazard ratio

  # Every coefficient comes with a p-value
  # The p-value represents the probability of observing this coefficient in a sample if the null hypothesis was true.
  # The null hypothesis states that the coefficient=0, meaning that the predictor variable does not influence the hazard rate (occurence of the event)

  # The CoxPHFitter computes p-values using the chi-squared test.
  # The reference is in "Survival Analysis by John P. Klein and Melvin L. Moeschberger, Second Edition", page 256

  p_values = cph._compute_p_values()
  hazard_ratios = cph.hazard_ratios_.tolist()
  coefficients_ci = cph.confidence_intervals_
  print(p_values)
  print(hazard_ratios)
  print(coefficients_ci)
  print("p-value:", p_values[4])
  print("hazard-ratio:", hazard_ratios[4])

  return p_values[4], hazard_ratios[4]

In [10]:
for i in range (1, 2):
  data_frame = pd.read_csv("TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv", sep="\t")
  data_frame = data_frame[['age', 'OS', 'OS.time', 'stage_2', 'stage_3', 'stage_4']]

  #name_embedding = "VAE_Z_EM_Mean_{i}".format(i=i)
  name_embedding = "UMAP_EM"
  embeddings = np.loadtxt(name_embedding+".txt")
  print("Embeddings shape: ", embeddings.shape)
  print("------------- i =", i, "--------------------")
  number_of_clusters = 4
  ckm = CKmeans(k=number_of_clusters, n_rep=20, p_samp=1, p_feat=1)
  ckm.fit(embeddings)
  ckm_results = ckm.predict(embeddings, linkage_type='average')
  print(ckm_results.cl)
  consensus_matrix = ckm_results.cmatrix
  print(consensus_matrix)

  cluster_0 = []
  cluster_1 = []
  cluster_2 = []
  cluster_3 = []
  for j in range(0, ckm_results.cl.shape[0]):
    if ckm_results.cl[j] == 1:
      cluster_1.append(j)
    elif ckm_results.cl[j] == 2:
      cluster_2.append(j)
    elif ckm_results.cl[j] == 3:
      cluster_3.append(j)
    elif ckm_results.cl[j] == 0:
      cluster_0.append(j)
  #print(cluster_0)
  #print(cluster_1)
  #print(cluster_2)
  #print(cluster_3)
  print("Cluster sizes: ", len(cluster_0), ", ", len(cluster_1), ", ", len(cluster_2), ", ", len(cluster_3))
  print("Sum of cluster sizes: ", len(cluster_0)+len(cluster_1)+len(cluster_2)+len(cluster_3))

  cluster_0 = drop_elements_from_cluster(cluster_0, consensus_matrix)
  cluster_1 = drop_elements_from_cluster(cluster_1, consensus_matrix)
  cluster_2 = drop_elements_from_cluster(cluster_2, consensus_matrix)
  cluster_3 = drop_elements_from_cluster(cluster_3, consensus_matrix)
  print("Cluster sizes after dropping:", len(cluster_0), ", ", len(cluster_1), ", ", len(cluster_2), ", ", len(cluster_3),)

  cluster_column_list = convert_indices_to_binary_list(embeddings.shape[0], cluster_0, cluster_1, cluster_2, cluster_3)
  print("Column_lengths: ",  len(cluster_column_list[0]))

  for j in range(0, number_of_clusters):
    if cluster_column_list[j].count(1) > 10:
      print("Cluster size in column form:", cluster_column_list[j].count(1))
      p_value, hazard_ratio = fit_cox_model(cluster_column_list[j], data_frame)
      print("p-value:", p_value)
      print("hazard ratio:", hazard_ratio)
      cluster_name = "k_5_{base}_{j}".format(base=name_embedding, j=j)
      run = wandb.init(project="SURVIVAL_ANALAYSIS_k=4", # Set the project where this run will be logged
                       name=cluster_name
      )

      wandb.log({
                "p_value": p_value,
                "hazard_ratio": hazard_ratio,
                "cluster_size": cluster_column_list[j].count(1)
            })

      if p_value < 0.001:
        file_name = cluster_name + ".txt"
        np.savetxt(file_name, cluster_column_list[j])



Embeddings shape:  (1079, 10)
------------- i = 1 --------------------
[3 1 1 ... 1 1 1]
[[1.   0.   0.   ... 0.   0.   0.05]
 [0.   1.   1.   ... 1.   1.   0.85]
 [0.   1.   1.   ... 1.   1.   0.85]
 ...
 [0.   1.   1.   ... 1.   1.   0.85]
 [0.   1.   1.   ... 1.   1.   0.85]
 [0.05 0.85 0.85 ... 0.85 0.85 1.  ]]
Cluster sizes:  183 ,  223 ,  332 ,  341
Sum of cluster sizes:  1079
Cluster sizes after dropping: 181 ,  145 ,  79 ,  116
Column_lengths:  1079
Cluster size in column form: 181
Original dataframe shape:  (1079, 6)
       age   OS  OS.time  stage_2  stage_3  stage_4  cluster
0     55.0  0.0   4047.0      NaN      NaN      NaN        0
180   81.0  0.0    608.0      NaN      NaN      NaN        0
213   76.0  0.0   1217.0      NaN      NaN      NaN        0
222   76.0  0.0    304.0      NaN      NaN      NaN        0
223   40.0  0.0    304.0      NaN      NaN      NaN        0
225   69.0  0.0     31.0      NaN      NaN      NaN        0
235   68.0  0.0    579.0      NaN      Na

<lifelines.CoxPHFitter: fitted with 1054 total observations, 915 right-censored observations>
             duration col = 'OS.time'
                event col = 'OS'
      baseline estimation = breslow
   number of observations = 1054
number of events observed = 139
   partial log-likelihood = -748.62
         time fit was run = 2024-03-07 19:35:45 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.04       1.04       0.01             0.02             0.05                 1.02                 1.05
stage_2     0.60       1.82       0.28             0.05             1.15                 1.05                 3.16
stage_3     1.34       3.81       0.30             0.76             1.92                 2.14                 6.81
stage_4     2.45      11.64       0.37             1.73             3.18                 5.65                23.96
cluster     0.25       1.29       0.23            -0.21             0.71                 0.81                 2.04

            cmp to    z      p   -log2(p)
covariate                                
age           0.00 5.43 <0.005      24.09
stage_2       0.00 2.13   0.03       4.91
stage_3       0.00 4.52 <0.005      17.33
stage_4       0.00 6.66 <0.005      35.12
cluster       0.00 1.08   0.28       1.84
---
Concordance = 0.76
Partial AIC = 1507.25
log-likelihood ratio test = 76.68 on 5 df
-log2(p) of ll-ratio test = 47.78

wandb: Currently logged in as: cosybio-compsysmed. Use `wandb login --relogin` to force relogin


[5.61409987e-08 3.32138681e-02 6.08511394e-06 2.66919133e-11
 2.78740574e-01]
[1.0377348756585514, 1.8197070605073922, 3.814634860046608, 11.639785101419072, 1.289267250624694]
           95% lower-bound  95% upper-bound
covariate                                  
age               0.023672         0.050408
stage_2           0.047659         1.149692
stage_3           0.758732         1.918958
stage_4           1.732525         3.176333
cluster          -0.205673         0.713821
p-value: 0.2787405736610419
hazard-ratio: 1.289267250624694
p-value: 0.2787405736610419
hazard ratio: 1.289267250624694


Cluster size in column form: 145
Original dataframe shape:  (1079, 6)
       age   OS  OS.time  stage_2  stage_3  stage_4  cluster
0     55.0  0.0   4047.0      NaN      NaN      NaN        0
180   81.0  0.0    608.0      NaN      NaN      NaN        0
213   76.0  0.0   1217.0      NaN      NaN      NaN        0
222   76.0  0.0    304.0      NaN      NaN      NaN        0
223   40.0  0.0    304.0      NaN      NaN      NaN        0
225   69.0  0.0     31.0      NaN      NaN      NaN        0
235   68.0  0.0    579.0      NaN      NaN      NaN        0
397   43.0  1.0   3262.0      NaN      NaN      NaN        0
459   46.0  1.0    749.0      NaN      NaN      NaN        0
463   90.0  1.0   1542.0      NaN      NaN      NaN        0
470   45.0  1.0   2573.0      NaN      NaN      NaN        1
474   61.0  0.0   7777.0      NaN      NaN      NaN        0
479   57.0  1.0   2373.0      NaN      NaN      NaN        0
482   73.0  1.0   3126.0      NaN      NaN      NaN        0
488   58.0  1.0

...

cluster_size,▁
hazard_ratio,▁
p_value,▁
cluster_size,145
hazard_ratio,0.56496
p_value,0.05177


Cluster size in column form: 116
Original dataframe shape:  (1079, 6)
       age   OS  OS.time  stage_2  stage_3  stage_4  cluster
0     55.0  0.0   4047.0      NaN      NaN      NaN        1
180   81.0  0.0    608.0      NaN      NaN      NaN        0
213   76.0  0.0   1217.0      NaN      NaN      NaN        0
222   76.0  0.0    304.0      NaN      NaN      NaN        1
223   40.0  0.0    304.0      NaN      NaN      NaN        0
225   69.0  0.0     31.0      NaN      NaN      NaN        0
235   68.0  0.0    579.0      NaN      NaN      NaN        0
397   43.0  1.0   3262.0      NaN      NaN      NaN        0
459   46.0  1.0    749.0      NaN      NaN      NaN        1
463   90.0  1.0   1542.0      NaN      NaN      NaN        1
470   45.0  1.0   2573.0      NaN      NaN      NaN        0
474   61.0  0.0   7777.0      NaN      NaN      NaN        0
479   57.0  1.0   2373.0      NaN      NaN      NaN        0
482   73.0  1.0   3126.0      NaN      NaN      NaN        0
488   58.0  1.0

<lifelines.CoxPHFitter: fitted with 1054 total observations, 915 right-censored observations>
             duration col = 'OS.time'
                event col = 'OS'
      baseline estimation = breslow
   number of observations = 1054
number of events observed = 139
   partial log-likelihood = -747.87
         time fit was run = 2024-03-07 19:35:58 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.04       1.04       0.01             0.02             0.05                 1.02                 1.05
stage_2     0.59       1.80       0.28             0.04             1.14                 1.04                 3.12
stage_3     1.31       3.72       0.30             0.73             1.89                 2.08                 6.63
stage_4     2.43      11.33       0.37             1.71             3.15                 5.51                23.28
cluster    -0.45       0.64       0.30            -1.03             0.13                 0.36                 1.14

            cmp to     z      p   -log2(p)
covariate                                 
age           0.00  5.53 <0.005      24.92
stage_2       0.00  2.09   0.04       4.78
stage_3       0.00  4.44 <0.005      16.79
stage_4       0.00  6.61 <0.005      34.56
cluster       0.00 -1.53   0.13       2.98
---
Concordance = 0.75
Partial AIC = 1505.75
log-likelihood ratio test = 78.18 on 5 df
-log2(p) of ll-ratio test = 48.82

[3.14261335e-08 3.64805565e-02 8.83598492e-06 3.94319528e-11
 1.26584883e-01]
[1.0385315843490837, 1.7995442123065928, 3.7160943217021876, 11.329483861939893, 0.635875864109376]
           95% lower-bound  95% upper-bound
covariate                                  
age               0.024416         0.051200
stage_2           0.036958         1.138108
stage_3           0.733719         1.891628
stage_4           1.707231         3.147586
cluster          -1.033607         0.128103
p-value: 0.12658488349699898
hazard-ratio: 0.635875864109376
p-value: 0.12658488349699898
hazard ratio: 0.635875864109376


cluster_size,▁
hazard_ratio,▁
p_value,▁
cluster_size,79
hazard_ratio,1.46131
p_value,0.18181
